In [18]:
import numpy as np
import math

f = open('hw8_movies.txt','r')
movie_names = f.read().splitlines()

f = open('hw8_ids.txt','r')
ids = f.read().splitlines()

f = open('hw8_ratings.txt','r')
ratings = f.read().splitlines()
ratings = np.genfromtxt('hw8_ratings.txt',missing_values = '?')
#create a mask for the ratings
ratings_mask = np.ones(ratings.shape)
ratings_mask[np.where(np.isnan(ratings))] = 0

In [19]:
ratings.shape

(258, 76)

#### 8.1

A)

In [20]:
mean_ratings = np.zeros((ratings.shape[1],1))
for j in range(ratings.shape[1]):
    observed = np.where(ratings_mask[:,j]==1)
    mean_ratings[j] = np.mean(ratings[observed,j])

In [21]:
movies_n_ratings = zip(movie_names,mean_ratings)
sort_ratings = sorted(movies_n_ratings, key = lambda y:y[1])[::-1]

In [22]:
for elem in sort_ratings:
    print(elem[0],np.round(elem[1],3)[0])

Inception 0.986
Interstellar 0.935
Three_Billboards_Outside_Ebbing 0.908
Django_Unchained 0.904
The_Martian 0.901
The_Dark_Knight_Rises 0.901
The_Theory_of_Everything 0.896
Black_Swan 0.891
Shutter_Island 0.879
Hidden_Figures 0.875
Avengers:_Infinity_War 0.869
The_Help 0.867
12_Years_a_Slave 0.863
The_Avengers 0.86
Ready_Player_One 0.857
Avengers:_Endgame 0.851
Les_Miserables 0.847
Parasite 0.845
The_Girls_with_the_Dragon_Tattoo 0.839
Now_You_See_Me 0.838
Joker 0.833
The_Lion_King 0.831
The_Social_Network 0.824
Gone_Girl 0.821
Harry_Potter_and_the_Deathly_Hallows:_Part_2 0.819
Wolf_of_Wall_Street 0.814
Room 0.81
Harry_Potter_and_the_Deathly_Hallows:_Part_1 0.804
Iron_Man_2 0.802
21_Jump_Street 0.795
Spiderman:_Far_From_Home 0.795
Ex_Machina 0.791
Her 0.791
La_La_Land 0.783
Frozen 0.776
Drive 0.775
X-Men:_First_Class 0.773
Midnight_in_Paris 0.769
Captain_America:_The_First_Avenger 0.765
Toy_Story_3 0.763
Darkest_Hour 0.761
Dunkirk 0.745
The_Great_Gatsby 0.741
The_Hateful_Eight 0.738
The

In [23]:
k = 4
num_iter = 256
z_init = np.genfromtxt('hw8_probZ_init.txt')
R_init = np.genfromtxt('hw8_probR_init.txt')
P_R = R_init # P(R|Z=zi)
P_Z = z_init

# columns of ratings refer to each movie
# rows are each students

T = ratings.shape[0]
T
t= 5
omega_t = np.where(ratings_mask[t,:]==1)
t_data  = ratings[t,omega_t].flatten()
P_R_Z = P_R[omega_t,3].flatten()

In [24]:
P_R.shape

(76, 4)

In [25]:
def calc_likelihood(ratings,ratings_mask,P_R,P_Z):
    L = 0
    
    for t in range(T):
        # provides the indices of the t_th students movies
        # that they have watched
        omega_t = np.where(ratings_mask[t,:]==1)
        t_data  = ratings[t,omega_t].flatten()
        inner_z_sum = 0
        #k is each column in P_R - z == k
        for i in range(k):
            # P(R_j = 1|Z=i) - j in omega_t
            P_R_Z = P_R[omega_t,i].flatten()
            
            inner_prod = P_Z[i] # initialize the product term with the P_Z = i
            for j in range(len(P_R_Z)):
                
                # inside the pi term -> [P(R=1|Z)^R_j]*[1-P(R=1|Z)^(1-R_j)]
                Pi_term = (P_R_Z[j]**t_data[j])*((1-P_R_Z[j])**(1-t_data[j]))
                inner_prod = inner_prod*Pi_term
            
            inner_z_sum+= inner_prod
        L += math.log(inner_z_sum)
    return L/T
            
                
                
                
                
    

In [26]:
calc_likelihood(ratings,ratings_mask,P_R,P_Z)

-29.327593818611266

In [27]:
def calc_p_it(ratings,ratings_mask,P_R,P_Z,T,k):
    
    '''
    returns a tx4 array of the p_it values for all t students
    
    calculates the probability Z = i given the visible ratings 
    in the t_th students data
    
    loops:
    
    over t samples
        
        over k - z values
            
            calculate P(Z|R)
        
        over k - z values
            
            calculate P(Z|R)/sum(P(Z|R)
            
            
    VARS:
    
    input:
    
        ratings: [num_students,num_movies]
        P_R    : [num_movies,num_Z] P(R|Z) for each movie for a given z value
        P_Z    : [num_z] - probabilities of each of the z_values
    
    intermediate:
        omega_t: gives the index of the visible movie ratings for the t_th student
        t_data: the visible rating data for the t_th student
        P_Z_R_numerator[num_z] stores the numerator value for P(Z|R) for all Z_i [z1,z2,z3,z4]
        numerator: stores intermediate value of the numerator as it loops througha all t entries
        P_R_Z: the P(R=1|Z=i) for each set of observed data in t_th students ratings
        P_Z_R_denominator
        
        
        
    
    OUTPUT: p_it[num_students,num_z]
    '''


    p_it = np.zeros((T,k))
    
    for t in range(T):
        
        # provides the indices of the t_th students movies
        # that they have watched
        omega_t = np.where(ratings_mask[t,:]==1)
        t_data  = ratings[t,omega_t].flatten()
        P_Z_R_numerator = []
        for i in range(k):
            
            # P(R_j = 1|Z=i) - j in omega_t
            P_R_Z = P_R[omega_t,i].flatten()
            # initialize the product term with the P_Z = i
            numerator = P_Z[i] 
            for j in range(len(P_R_Z)):
                # inside the pi term -> [P(R=1|Z)^R_j]*[1-P(R=1|Z)^(1-R_j)]
                Pi_term = (P_R_Z[j]**t_data[j])*((1-P_R_Z[j])**(1-t_data[j]))
                numerator = numerator*Pi_term
        
            P_Z_R_numerator.extend([numerator])
            
        P_Z_R_denominator = sum(P_Z_R_numerator)
        for i in range(k):
            p_it[t,i] = P_Z_R_numerator[i]/P_Z_R_denominator

    return p_it
    

In [28]:
def update_routine(ratings,ratings_mask,P_R,P_Z,T,k,p_it):
    '''
    update the Z_i values
    P(Z=i) = 1/T*sum(p_it)
    p_it = P(Z|R_j(t)) for all t students
    
    '''
    new_Z = []
    for i in range(k):
        new_Z.extend([np.sum(p_it[:,i])/T])

    ### Calc new P(R|Z) ###
    new_P_R_Z = np.zeros(P_R.shape)
    for i in range(k): # possible z values
        for j in range(ratings.shape[1]): # movies
            numerator = 0
            denominator = np.sum(p_it[:,i])
            for t in range(T): # samples
                if ratings[t,j] == 1 or ratings[t,j] == 0:
                    if ratings[t,j] == 1:
                        numerator += p_it[t,i]
                else:
                    numerator += p_it[t,i]*P_R[j,i]
            
            new_P_R_Z[j,i] = numerator/denominator
    
    return new_Z,new_P_R_Z

In [30]:
# main_loop
k = 4
num_iter = 256
z_init = np.genfromtxt('hw8_probZ_init.txt')
R_init = np.genfromtxt('hw8_probR_init.txt')
P_R = R_init # P(R|Z=zi)
P_Z = z_init

# columns of ratings refer to each movie
# rows are each students

T = ratings.shape[0]
i_list = [0,1,2,4,16,32,64,128,256]
for i in range(num_iter+1):
    L = calc_likelihood(ratings,ratings_mask,P_R,P_Z)
    if i in i_list:
        print(i,': ',np.round(L,4))
    
    p_it = calc_p_it(ratings,ratings_mask,P_R,P_Z,T,k)
    P_Z,P_R = update_routine(ratings,ratings_mask,P_R,P_Z,T,k,p_it)
    
    

0 :  -29.3276
1 :  -18.1393
2 :  -16.1713
4 :  -14.9416
16 :  -13.8581
32 :  -13.764
64 :  -13.7398
128 :  -13.7377
256 :  -13.7375


F)

In [40]:
Steve_Ratings = ratings[239,:]

In [81]:
#### CALC P(R_l=1:Rj) ###
omega_unseen_steve =np.where(ratings_mask[239,:]==0)
pi_steve    = p_it[239,:] # all of the (P=Zi|Rj) [4,0]
P_R_unseen  = P_R[omega_unseen_steve[0],:]         #[#,4]
P_Rl_True = np.zeros((omega_unseen_steve[0].shape[0],))    #

for l in range(omega_unseen_steve[0].shape[0]):
    placeholder = 0
    for i in range(k):
        placeholder = placeholder+(P_R_unseen[l,i]*pi_steve[i])
        #print(placeholder)
    P_Rl_True[l] = placeholder
    
unseen_index = omega_unseen_steve[0].tolist()
unseen_titles = [movie_names[i] for i in unseen_index]
# unseen_movies = movie_names[omega_unseen_steve[0].tolist()] 

In [84]:
sort_ratings

[('Hidden_Figures', 0.998303981032741),
 ('The_Hateful_Eight', 0.9980120121228705),
 ('The_Farewell', 0.9970826037255528),
 ('Shutter_Island', 0.9797645631010524),
 ('The_Martian', 0.9756769589221221),
 ('Ready_Player_One', 0.9647765979861145),
 ('Spiderman:_Far_From_Home', 0.9644760566760855),
 ('12_Years_a_Slave', 0.9637645648109957),
 ('Drive', 0.9369468323289725),
 ('The_Help', 0.9367223404048632),
 ('Her', 0.9308202953145637),
 ('Gone_Girl', 0.9271705056429871),
 ('Black_Swan', 0.9191688889373434),
 ('Les_Miserables', 0.918843465369854),
 ('The_Theory_of_Everything', 0.9122517072674587),
 ('Frozen', 0.9075759614806203),
 ('Joker', 0.8962632590290507),
 ('Chappaquidick', 0.896013583224202),
 ('Three_Billboards_Outside_Ebbing', 0.8871603393748463),
 ('The_Revenant', 0.8812629261923193),
 ('The_Lion_King', 0.879466607948499),
 ('La_La_Land', 0.8790273799909772),
 ('The_Girls_with_the_Dragon_Tattoo', 0.86753897506893),
 ('X-Men:_First_Class', 0.8578737100415335),
 ('Captain_America:_T

In [85]:
unseen_n_ratings = zip(unseen_titles,P_Rl_True)
sort_ratings = sorted(unseen_n_ratings, key = lambda y:y[1])[::-1]

for elem in sort_ratings:
    print(elem[0],np.round(elem[1],3))

Hidden_Figures 0.998
The_Hateful_Eight 0.998
The_Farewell 0.997
Shutter_Island 0.98
The_Martian 0.976
Ready_Player_One 0.965
Spiderman:_Far_From_Home 0.964
12_Years_a_Slave 0.964
Drive 0.937
The_Help 0.937
Her 0.931
Gone_Girl 0.927
Black_Swan 0.919
Les_Miserables 0.919
The_Theory_of_Everything 0.912
Frozen 0.908
Joker 0.896
Chappaquidick 0.896
Three_Billboards_Outside_Ebbing 0.887
The_Revenant 0.881
The_Lion_King 0.879
La_La_Land 0.879
The_Girls_with_the_Dragon_Tattoo 0.868
X-Men:_First_Class 0.858
Captain_America:_The_First_Avenger 0.858
Toy_Story_3 0.855
Parasite 0.845
The_Perks_of_Being_a_Wallflower 0.843
Rocketman 0.841
Darkest_Hour 0.84
Room 0.812
Prometheus 0.803
Midnight_in_Paris 0.801
Phantom_Thread 0.799
Dunkirk 0.788
Terminator:_Dark_Fate 0.762
The_Hunger_Games 0.76
Manchester_by_the_Sea 0.757
American_Hustle 0.756
Once_Upon_a_Time_in_Hollywood 0.732
Pokemon_Detective_Pikachu 0.724
Us 0.721
Fast_Five 0.721
Pitch_Perfect 0.703
Fast_&_Furious:_Hobbs_&_Shaw 0.695
Man_of_Steel 0.